# Date : 25-09-2022
codegrepper.com/code-examples/python/gridsearchcv+get+best+model
# Automatically find best imputation parameters for a Model using Cross Validation
*   Data set: Diabeteis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
import scipy.stats as stats
import seaborn as sn
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.impute  import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
cols_to_use = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
    'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea',
    'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
    'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
    'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
    'WoodDeckSF',  'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
    'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold',
    'SalePrice'
]

In [3]:
data=pd.read_csv("/content/kaggle_house_pred_train.csv", usecols=cols_to_use)

In [24]:
cols_to_use.remove('SalePrice')
X_train,X_test,y_train,y_test=train_test_split(data[cols_to_use],
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

# Multivariate 

# Setup Pipeline

In [6]:
pipeline=Pipeline(steps=[('imputer',KNNImputer(
                          n_neighbors=5,
                          weights='distance',
                          add_indicator=False)),
                          ('scaler', StandardScaler()),
                          ('regressor', Lasso(max_iter=2000))
                         ]
                   )

#Create the grid with all the parameters that we would like to test


In [11]:
param_grid={
    'imputer__n_neighbors':[3,5,10,20],
    'imputer__weights':['uniform','distance'],
    'imputer__add_indicator':[True,False],
    'regressor__alpha':[10,100,200]
       }
grid_search=GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='r2')

In [15]:
grid_search.fit(X_train, y_train)
print("Best Score is ",grid_search.score(X_train, y_train))

Best Score is  0.8447825313902115


In [13]:
grid_search.best_estimator_


Pipeline(steps=[('imputer',
                 KNNImputer(add_indicator=True, n_neighbors=10,
                            weights='distance')),
                ('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=200, max_iter=2000))])

In [18]:
grid_search.best_params_

{'imputer__add_indicator': True,
 'imputer__n_neighbors': 10,
 'imputer__weights': 'distance',
 'regressor__alpha': 200}

In [19]:
grid_search.best_score_

0.8137245727766841

In [17]:
print("Best Score is ",grid_search.score(X_test, y_test))

Best Score is  0.7299614753611061


# Univariate

# SetUp Pipeline

In [20]:
pipeline=Pipeline(steps=[('imputer',SimpleImputer(strategy='mean', fill_value=-1)),
                          ('scaler', StandardScaler()),
                          ('regressor', Lasso(max_iter=2000))
                         ]
                   )

# Setup Parameters Grid

In [25]:
parameters_grid={
               'imputer__strategy':['mean','median','constant'],
               'imputer__add_indicator': [True, False],
               'regressor__alpha':[10,100,200]
}
grid_search=GridSearchCV(pipeline,parameters_grid,cv=5, n_jobs=-1, scoring='r2')

In [26]:
grid_search.fit(X_train,y_train)
print("Best Score",grid_search.score(X_train,y_train))

Best Score 0.8450278523645806


# Best estimator

In [27]:
grid_search.best_estimator_

Pipeline(steps=[('imputer', SimpleImputer(fill_value=-1, strategy='constant')),
                ('scaler', StandardScaler()),
                ('regressor', Lasso(alpha=200, max_iter=2000))])

# Best Parameter

In [28]:
grid_search.best_params_

{'imputer__add_indicator': False,
 'imputer__strategy': 'constant',
 'regressor__alpha': 200}

# Best Score

In [29]:
grid_search.best_score_

0.8141840732084573

In [30]:
print("Best Tets ",grid_search.score(X_test,y_test))

Best Tets  0.728986724909142
